# AUTOMATIC GENERATION OF MDA ANALYSIS FROM REQUIREMENTS

**First step is to search the Papyrus Requirements model:**

In [1]:
import shutil
import os

#Make no attention to the "r" character before the directory path's examples. It is only a python syntax so the "\" character can be taken literally.

dir_or = input("Please enter the papyrus model directory path:\n")
# During the developement period the path used was the one from the Papyrus workspace. 
# See example below:
# r'C:\Users\rroja\workspace-papyrus\Sellar\Sellar.uml'
# C:\Users\rroja\Documents\ISAE\Research Project\Papyrus_models\UAV_tools_test_model\UAV_tools_test.uml

dir_target = input("Please crate a directory path to copy the original model file for data management and thus avoid any damage to the model source code.\n"+
                   "Once created please input it below:\n (Please notice that results from OpenMDAO will be stored in a folder created automatically in this directory\n")
#As dir_target the path used was as below. It can be any preferred location.
#dir_target = r'C:\Users\rroja\Documents\ISAE\Research Project\Jupyter_Tests\Copied_uml-to-xml'

#The name of the generated copy is with a direct parse to XML file. model_XML.xml
name='model_XML.xml'
path = os.path.join(dir_target, name)

shutil.copyfile(dir_or, path)

Please enter the papyrus model directory path:
C:\Users\rroja\Documents\ISAE\Research Project\Papyrus_models\UAV_tools_test_model\UAV_tools_test.uml
Please crate a directory path to copy the original model file for data management and thus avoid any damage to the model source code.
Once created please input it below:
 (Please notice that results from OpenMDAO will be stored in a folder created automatically in this directory
C:\Users\rroja\Documents\ISAE\Research Project\Jupyter_Tests\Copied_uml-to-xml


'C:\\Users\\rroja\\Documents\\ISAE\\Research Project\\Jupyter_Tests\\Copied_uml-to-xml\\model_XML.xml'

**The xml file is converted to a json file:**

In [2]:
import json
import xmltodict

with open(r'C:\Users\rroja\Documents\ISAE\Research Project\Jupyter_Tests\Copied_uml-to-xml\model_XML.xml') as xml_file:
    data_dict = xmltodict.parse(xml_file.read())
    xml_file.close()

    json_data = json.dumps(data_dict)
    with open("data.json", "w") as json_file:
        json_file.write(json_data)
        json_file.close()

**The file is read:**

In [3]:
# JSON file
f = open ('data.json', "r")
  
# Reading from file
data = json.loads(f.read())
data

{'xmi:XMI': {'@xmi:version': '20131001',
  '@xmlns:xmi': 'http://www.omg.org/spec/XMI/20131001',
  '@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
  '@xmlns:Requirementprofile': 'http:///schemas/Requirementprofile/_nfUwUI9QEeyDkrtcFBe8AA/10',
  '@xmlns:ecore': 'http://www.eclipse.org/emf/2002/Ecore',
  '@xmlns:uml': 'http://www.eclipse.org/uml2/5.0.0/UML',
  '@xsi:schemaLocation': 'http:///schemas/Requirementprofile/_nfUwUI9QEeyDkrtcFBe8AA/10 ../Requirement%20profile/Requirement%20profile.profile.uml#_nfVXYI9QEeyDkrtcFBe8AA',
  'uml:Model': {'@xmi:id': '_Fwf68C0IEeydKKxpjt4sqQ',
   '@name': 'Sellar',
   'packageImport': [{'@xmi:type': 'uml:PackageImport',
     '@xmi:id': '_F1PRAC0IEeydKKxpjt4sqQ',
     'importedPackage': {'@xmi:type': 'uml:Model',
      '@href': 'pathmap://UML_LIBRARIES/UMLPrimitiveTypes.library.uml#_0'}},
    {'@xmi:type': 'uml:PackageImport',
     '@xmi:id': '_F1ehkC0IEeydKKxpjt4sqQ',
     'importedPackage': {'@xmi:type': 'uml:Package',
      '@href': 'pat

**A dictionary with requirements' xmi:id as keys and id as values:**

It is performend for all requirements so we can later retrieve the actual requirements id liked to MDAO analysis and the corresponding quantity of interest (QoI).

The origin of this step comes from the fact the a requirement name might be different than the given requirement id or variable. Therefore, the Papyrus model' source code gives an indentifier to link each block to its attributes. Thus, the algorithm serves of it to automatically retrieve the wanted information (variable Id towards MDAO and QoI).

In [4]:
#Constructing the dictionary that gatter the Ids liked to MDAO with the internal papyrus @xmi:id.
id_dict={}

for Requirements in data["xmi:XMI"]["Requirementprofile:RequirementPlus"]:
    id_dict[Requirements.get("@base_NamedElement")]=Requirements.get("@MDAO_id")    

id_dict

{'_Dv4fUC0OEey2VL7OA8AAEg': None,
 '_I00Y0C0OEey2VL7OA8AAEg': None,
 '_Nef5sC0OEey2VL7OA8AAEg': '',
 '_nrpjEC0OEey2VL7OA8AAEg': 'MTOW',
 '_t65TkC0OEey2VL7OA8AAEg': ''}

**A dictionary with xmi:id as keys and the requirement block name as values**

In [5]:
#Constructing the names vs xmi:id dictionary. 
#It will work for simple papyrus models, if ever a complex diagram use the algorithm after parameters diagram.
name_dict={}
for req_block in data["xmi:XMI"]["uml:Model"]['packagedElement']:
    if req_block.get('@xmi:type')=='uml:Package':
        if type(req_block['packagedElement']) == dict:
            name_dict[req_block['packagedElement']['@xmi:id']]=req_block['packagedElement']['@name']
        else:
            for variable in req_block['packagedElement']:
                name_dict[variable.get('@xmi:id')]=variable.get("@name")

name_dict

{'_Dv4fUC0OEey2VL7OA8AAEg': 'x',
 '_I00Y0C0OEey2VL7OA8AAEg': 'z1',
 '_Nef5sC0OEey2VL7OA8AAEg': 'Empennage Weight',
 '_nrpjEC0OEey2VL7OA8AAEg': 'g1',
 '_t65TkC0OEey2VL7OA8AAEg': 'g2'}

In [6]:
name_dict

{'_Dv4fUC0OEey2VL7OA8AAEg': 'x',
 '_I00Y0C0OEey2VL7OA8AAEg': 'z1',
 '_Nef5sC0OEey2VL7OA8AAEg': 'Empennage Weight',
 '_nrpjEC0OEey2VL7OA8AAEg': 'g1',
 '_t65TkC0OEey2VL7OA8AAEg': 'g2'}

**Creation of the Requirement's dictionary (id: name)**

In [7]:
#Code to automatically create the requirements dictionary 
#The aim is to verify that the correlation between the information is correct.
Requirements_dict = {}
for xmi_id in id_dict:
    if xmi_id in name_dict.keys():
        
         Requirements_dict[id_dict[xmi_id]]=name_dict[xmi_id]
print(Requirements_dict)

{None: 'z1', '': 'g2', 'MTOW': 'g1'}


**Generation of a dictionary storing the numerical values of each requirement:**

In [8]:
# WE SHOULD ADDRESS ONLY THE NEEDED PARAMETERS FROM THE MDAO PROBLEM.
# Below we get all of them that have a "QoI" just to code the case when a req. has not this attribute. 
# This algorithm is thinked mainly for troubleshooting and data analysis.

Parameters_dict = {}
for Requirements in data["xmi:XMI"]["Requirementprofile:RequirementPlus"]:    
    if Requirements.get("QoI")==None:
        print("no quantity of interest")
    else:
        if Requirements.get("@MDAO_id") in Requirements_dict.keys():
            Parameters_dict[Requirements.get("@MDAO_id")] = Requirements["QoI"]

print(Parameters_dict)

no quantity of interest
no quantity of interest
no quantity of interest
no quantity of interest
{'MTOW': '20.0'}


In [9]:
#THIS CODE REMPLIES ALL THE REQUIREMENTS DICTIONARY AND PARAMETERS DICTIONARY GENERATION FOR A MORE DIRECT GENERATION.
# It does not depend on the papyrus model complexity but in the naming of the stereotype profile extention.

P_dict={}
for Requirements in data["xmi:XMI"]["Requirementprofile:RequirementPlus"]:
    if Requirements.get("QoI")==None or Requirements.get("@MDAO_id")==None :
        P_dict[Requirements.get("@MDAO_id")]=Requirements.get("QoI")
P_dict

{None: None, '': None}

**WhatsOpt log in:**

In [10]:
# Completely necessary
!pip install -U wop

Requirement already up-to-date: wop in c:\users\rroja\anaconda3\lib\site-packages (2.0.0)


In [11]:
#!wop login https://ether.onera.fr/whatsopt

from whatsopt.whatsopt_client import WhatsOpt
wop = WhatsOpt(url="https://ether.onera.fr/whatsopt")
ok = wop.login(echo=True)

Successfully logged into WhatsOpt (https://ether.onera.fr/whatsopt)



In [12]:
!wop status
#!wop list

You are logged in https://ether.onera.fr/whatsopt
No local analysis found
  (use 'wop list' and 'wop pull <id>' to retrieve an existing analysis)
  (use 'wop push <analysis.py>' to push from the local OpenMDAO code to the server)



**Pull of the project storing all the disciplines:**

In [13]:
Project_Pull = !wop pull --json --project-id 14

Project_Pull_str=Project_Pull[0]

with open("Project_Pull_def.json", "w") as json_file:
    json_file.write(Project_Pull_str)
    json_file.close()
    
import json
# Disciplines JSON file
p = open ('Project_Pull_def.json', "r")
Project_Disc = json.loads(p.read())

print(Project_Disc)

{'name': 'UAV_tools_test', 'created_at': '2022-02-16T16:45:21.813Z', 'owner_email': 'rafael.rojas-cardenas-@student.isae-supaero.fr', 'description': 'Application of methodology on MDA for UAV', 'analyses_attributes': [{'name': 'Masses_Analysis', 'disciplines_attributes': [{'name': '__DRIVER__', 'type': 'null_driver', 'variables_attributes': [{'name': 'Empennage_weight', 'io_mode': 'out', 'shape': '1', 'type': 'Float', 'desc': '', 'units': '', 'active': True, 'distributions_attributes': []}, {'name': 'Endurance', 'io_mode': 'out', 'shape': '1', 'type': 'Float', 'desc': '', 'units': '', 'active': True, 'distributions_attributes': []}, {'name': 'Fuselage_weight', 'io_mode': 'out', 'shape': '1', 'type': 'Float', 'desc': '', 'units': '', 'active': True, 'distributions_attributes': []}, {'name': 'MTOW', 'io_mode': 'in', 'shape': '1', 'type': 'Float', 'desc': '', 'units': '', 'active': True, 'distributions_attributes': []}, {'name': 'Nb_engines', 'io_mode': 'out', 'shape': '1', 'type': 'Float

**Generation of a list containing all the disciplines from the project:**

In [14]:
#Creation of the list for all the sub-analysis (Disciplines) in the whatsopt project
Project_Disc_list = []

In [15]:
#Disciplines parse
for disciplines in Project_Disc['analyses_attributes']:
    Project_Disc_list.append(disciplines)

Project_Disc_list

[{'name': 'Masses_Analysis',
  'disciplines_attributes': [{'name': '__DRIVER__',
    'type': 'null_driver',
    'variables_attributes': [{'name': 'Empennage_weight',
      'io_mode': 'out',
      'shape': '1',
      'type': 'Float',
      'desc': '',
      'units': '',
      'active': True,
      'distributions_attributes': []},
     {'name': 'Endurance',
      'io_mode': 'out',
      'shape': '1',
      'type': 'Float',
      'desc': '',
      'units': '',
      'active': True,
      'distributions_attributes': []},
     {'name': 'Fuselage_weight',
      'io_mode': 'out',
      'shape': '1',
      'type': 'Float',
      'desc': '',
      'units': '',
      'active': True,
      'distributions_attributes': []},
     {'name': 'MTOW',
      'io_mode': 'in',
      'shape': '1',
      'type': 'Float',
      'desc': '',
      'units': '',
      'active': True,
      'distributions_attributes': []},
     {'name': 'Nb_engines',
      'io_mode': 'out',
      'shape': '1',
      'type': 'Float'

**Generation of a list of all the disciplines whose outputs are found in the requirements:**

In [16]:
#Discipline List append algorithm for the outputs variables that are found in the req. diagram.
MDA_Disc_list = []

for Disc in Project_Disc_list:
    for disc_attr in Disc["disciplines_attributes"]:
        if disc_attr['name']+'_Analysis' == Disc['name']:
            for var_attr in disc_attr['variables_attributes']:
                if var_attr['name'] in Parameters_dict.keys() and var_attr['io_mode'] == 'out':
                    if disc_attr['name'] not in MDA_Disc_list:
                        MDA_Disc_list.append(Disc)

                    
MDA_Disc_list


[{'name': 'Masses_Analysis',
  'disciplines_attributes': [{'name': '__DRIVER__',
    'type': 'null_driver',
    'variables_attributes': [{'name': 'Empennage_weight',
      'io_mode': 'out',
      'shape': '1',
      'type': 'Float',
      'desc': '',
      'units': '',
      'active': True,
      'distributions_attributes': []},
     {'name': 'Endurance',
      'io_mode': 'out',
      'shape': '1',
      'type': 'Float',
      'desc': '',
      'units': '',
      'active': True,
      'distributions_attributes': []},
     {'name': 'Fuselage_weight',
      'io_mode': 'out',
      'shape': '1',
      'type': 'Float',
      'desc': '',
      'units': '',
      'active': True,
      'distributions_attributes': []},
     {'name': 'MTOW',
      'io_mode': 'in',
      'shape': '1',
      'type': 'Float',
      'desc': '',
      'units': '',
      'active': True,
      'distributions_attributes': []},
     {'name': 'Nb_engines',
      'io_mode': 'out',
      'shape': '1',
      'type': 'Float'

**Algorithm to find the disciplines whose outputs are inputs for the disciplines previoulsy found. This algorithm is a loop that finishes when there are no more disciplines to be added to the list.**

In [17]:
# Searching for inputs in the disciplines that are outputs from others

Inputs_list = []

while(True):

    MDA_list_size_init= len(MDA_Disc_list)

    for Disc in MDA_Disc_list:
        for disc_attr in Disc["disciplines_attributes"]:
            if disc_attr['name']+'_Analysis' == Disc['name']:
                for var_attr in disc_attr['variables_attributes']:
                    if var_attr['io_mode'] == 'in':
                        if var_attr['name'] not in Inputs_list:
                            Inputs_list.append(var_attr['name'])

#Inputs_list

    for Disc in Project_Disc_list:
        for disc_attr in Disc["disciplines_attributes"]:
            if disc_attr['name']+'_Analysis' == Disc['name']:
                for var_attr in disc_attr['variables_attributes']:
                    if var_attr['io_mode'] == 'out':
                        if (var_attr['name'] in Inputs_list) and (Disc not in MDA_Disc_list):
                            MDA_Disc_list.insert(0,Disc)
                
    MDA_list_size_up= len(MDA_Disc_list)

    if MDA_list_size_up != MDA_list_size_init:
        MDA_list_size_init = MDA_list_size_up
        continue
    else:
        break        

In [18]:
MDA_Disc_list

[{'name': 'Mission_Analysis',
  'disciplines_attributes': [{'name': '__DRIVER__',
    'type': 'null_driver',
    'variables_attributes': [{'name': 'Endurance',
      'io_mode': 'in',
      'shape': '1',
      'type': 'Float',
      'desc': '',
      'units': '',
      'active': True,
      'distributions_attributes': []},
     {'name': 'Range',
      'io_mode': 'out',
      'shape': '1',
      'type': 'Float',
      'desc': '',
      'units': '',
      'active': True,
      'distributions_attributes': []},
     {'name': 'Speed',
      'io_mode': 'out',
      'shape': '1',
      'type': 'Float',
      'desc': '',
      'units': '',
      'active': True,
      'distributions_attributes': []},
     {'name': 'Wind_speed',
      'io_mode': 'out',
      'shape': '1',
      'type': 'Float',
      'desc': '',
      'units': '',
      'active': True,
      'distributions_attributes': []}]},
   {'name': 'Mission',
    'type': 'analysis',
    'variables_attributes': [{'name': 'Endurance',
      '

**Generation of the MDA json file to be pushed:
First a list of all the variables of the analysis**

In [19]:
list_of_variables=[]


#we start to append all "out" variables in the list without any change.
for Disc in MDA_Disc_list:
    for disc_attr in Disc["disciplines_attributes"]:
        if disc_attr['name']+'_Analysis' == Disc['name']:
            for var_attr in disc_attr['variables_attributes']:
                if var_attr['name'] in [v_dict['name'] for v_dict in list_of_variables]:
                    print('repeated variable')
                elif var_attr['io_mode'] == 'out':
                    list_of_variables.append(var_attr)
                        

repeated variable
repeated variable
repeated variable
repeated variable
repeated variable
repeated variable
repeated variable
repeated variable
repeated variable
repeated variable
repeated variable
repeated variable
repeated variable


In [20]:
   
# Now we have to search all the "in" variables for each discipline leaving outside all previous "out" variables.


for Disc in MDA_Disc_list:
    for disc_attr in Disc["disciplines_attributes"]:
        if disc_attr['name']+'_Analysis' == Disc['name']:
            for var_attr in disc_attr['variables_attributes']:
                if var_attr['name'] in [v_dict['name'] for v_dict in list_of_variables]:
                    print('repeated variable')
                else:
                    Inputs_list.append(var_attr)
                    list_of_variables.append(var_attr)
                
list_of_variables                  

repeated variable
repeated variable
repeated variable
repeated variable
repeated variable
repeated variable
repeated variable
repeated variable
repeated variable
repeated variable
repeated variable
repeated variable
repeated variable
repeated variable
repeated variable
repeated variable
repeated variable
repeated variable
repeated variable
repeated variable
repeated variable
repeated variable
repeated variable
repeated variable
repeated variable
repeated variable
repeated variable
repeated variable


[{'name': 'Endurance',
  'io_mode': 'out',
  'shape': '1',
  'type': 'Float',
  'desc': '',
  'units': '',
  'active': True,
  'distributions_attributes': []},
 {'name': 'Fuselage_area',
  'io_mode': 'out',
  'shape': '1',
  'type': 'Float',
  'desc': '',
  'units': '',
  'active': True,
  'distributions_attributes': []},
 {'name': 'Fuselage_weight',
  'io_mode': 'out',
  'shape': '1',
  'type': 'Float',
  'desc': '',
  'units': '',
  'active': True,
  'distributions_attributes': []},
 {'name': 'L_HT',
  'io_mode': 'out',
  'shape': '1',
  'type': 'Float',
  'desc': '',
  'units': '',
  'active': True,
  'distributions_attributes': []},
 {'name': 'L_VT',
  'io_mode': 'out',
  'shape': '1',
  'type': 'Float',
  'desc': '',
  'units': '',
  'active': True,
  'distributions_attributes': []},
 {'name': 'Chord',
  'io_mode': 'out',
  'shape': '1',
  'type': 'Float',
  'desc': '',
  'units': '',
  'active': True,
  'distributions_attributes': []},
 {'name': 'Cx',
  'io_mode': 'out',
  'shape

**The QoI is inserted to those variables that are found in the Parameters_dict.**

In [21]:
#Algorithm to modify the top level DRIVER variables.

init_value_template = {'init': None, 'lower': None, 'upper': None}


for var in list_of_variables:
    if var['name'] in Parameters_dict.keys():
        init_value_template['init']=Parameters_dict.get(var['name'])
        var['parameter_attributes']=init_value_template
        init_value_template = {'init': None, 'lower': None, 'upper': None}
        
list_of_variables

[{'name': 'Endurance',
  'io_mode': 'out',
  'shape': '1',
  'type': 'Float',
  'desc': '',
  'units': '',
  'active': True,
  'distributions_attributes': []},
 {'name': 'Fuselage_area',
  'io_mode': 'out',
  'shape': '1',
  'type': 'Float',
  'desc': '',
  'units': '',
  'active': True,
  'distributions_attributes': []},
 {'name': 'Fuselage_weight',
  'io_mode': 'out',
  'shape': '1',
  'type': 'Float',
  'desc': '',
  'units': '',
  'active': True,
  'distributions_attributes': []},
 {'name': 'L_HT',
  'io_mode': 'out',
  'shape': '1',
  'type': 'Float',
  'desc': '',
  'units': '',
  'active': True,
  'distributions_attributes': []},
 {'name': 'L_VT',
  'io_mode': 'out',
  'shape': '1',
  'type': 'Float',
  'desc': '',
  'units': '',
  'active': True,
  'distributions_attributes': []},
 {'name': 'Chord',
  'io_mode': 'out',
  'shape': '1',
  'type': 'Float',
  'desc': '',
  'units': '',
  'active': True,
  'distributions_attributes': []},
 {'name': 'Cx',
  'io_mode': 'out',
  'shape

In [22]:
#Algorithm to modify the local discipline Driver.

init_value_template = {'init': None, 'lower': None, 'upper': None}

for Disc in Project_Disc_list:
    for disc_attr in Disc["disciplines_attributes"]:
        if disc_attr['name'] == '__DRIVER__':
            for var_attr in disc_attr['variables_attributes']:
                if var_attr['name'] in Parameters_dict.keys():
                    init_value_template['init']=Parameters_dict.get(var_attr['name'])
                    var_attr['parameter_attributes']=init_value_template
                    init_value_template = {'init': None, 'lower': None, 'upper': None}
                    
#Project_Disc_list

**For the Driver, the io_mode needs to be switched:**
If the variables are an output from a discipline they shall be treated as "input" in the driver and viceversa.
Notice that a "new_list" is needed in order to make a "deep copy" and do not modify the original format."

In [23]:
import copy
new_list = copy.deepcopy(list_of_variables)
for instance in new_list:
    if instance['io_mode'] == 'out':
        instance['io_mode'] = 'in'
    elif instance['io_mode'] == 'in':
        instance['io_mode'] = 'out'
#new_list

**Insert into the json file the list of variables:**

First we start to generate the format for the DRIVER

In [24]:
DRONE_MDA_push = {'name': 'DRONE_Reference_push', 
             'disciplines_attributes': [{
                 'name': '__DRIVER__', 
                 'type': 'null_driver',
                 'variables_attributes':[]}]}

DRONE_MDA_push['disciplines_attributes'][0]['variables_attributes'] = new_list
#DRONE_MDA_push

**Insert the disciplines:**

In [25]:
for disc in MDA_Disc_list:
    DRONE_MDA_push['disciplines_attributes'].append({'name': disc['name'],'type': 'mda', 'sub_analysis_attributes': disc})

#DRONE_MDA_push

**Convert json file into a string so as to be pushed to WhatsOpt:**

In [26]:
string_problem_mod = json.dumps(DRONE_MDA_push)
Problem_json=[]
Problem_json.append(string_problem_mod)
#Problem_json

**Push into WhatsOpt:**

In [27]:
with open("problem_definition_mod.json", "w") as json_file:
    json_file.write(string_problem_mod)
    json_file.close()
!wop push --json problem_definition_mod.json

Analysis DRONE_Reference_push pushed


**Pulling the analysis to run it**

In [43]:
#Getting the last modified project id
owned_project_list = !wop list
last_project=owned_project_list[len(owned_project_list)-2]
lp_list=last_project.split(' ')
lp_id=lp_list[0]
lp_id

'1670'

In [44]:
#Pulling the last modified project
newpath = r''+dir_target+''+'\OpenMDAO_Files\\'+lp_id
if not os.path.exists(newpath):
    os.makedirs(newpath)
os.chdir(newpath)
!dir

 El volumen de la unidad C es Windows
 El número de serie del volumen es: DAFC-E9CF

 Directorio de C:\Users\rroja\Documents\ISAE\Research Project\Jupyter_Tests\Copied_uml-to-xml\OpenMDAO_Files\1670

09/03/2022  04:23 p. m.    <DIR>          .
09/03/2022  04:23 p. m.    <DIR>          ..
               0 archivos              0 bytes
               2 dirs  38,768,828,416 bytes libres


In [45]:
#How to automatically pull the lastest modification of the project.

cmd_lp = "wop pull " + lp_id

import subprocess
subprocess.run(cmd_lp, shell=True, check=True)

CompletedProcess(args='wop pull 1670', returncode=0)

In [46]:
%run run_mda.py

|  
|  =================
|  fuselage_analysis
|  =================
|  NL: NLBGS Converged in 1 iterations
|  
|  =============
|  aero_analysis
|  =============
|  NL: NLBGS Converged in 1 iterations
|  
|  ==================
|  empennage_analysis
|  ==================
|  NL: NLBGS Converged in 2 iterations
|  
|  =================
|  fuselage_analysis
|  =================
|  NL: NLBGS Converged in 1 iterations
|  
|  =============
|  aero_analysis
|  =============
|  NL: NLBGS Converged in 1 iterations
|  
|  ==================
|  empennage_analysis
|  ==================
|  NL: NLBGS Converged in 1 iterations
NL: NLBGS Converged in 2 iterations
9 Input(s) in 'model'

varname              val 
-------------------  ----
fuselage_analysis
  fuselage
    Fuselage_length  [1.]
    Speed            [1.]
aero_analysis
  aero
    Speed            [1.]
    Wing_span        [1.]
empennage_analysis
  empennage
    Chord            [1.]
    L_HT             [1.]
    L_VT             [1.]
    Surf

In [99]:
#!wop -h